In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

tf.get_logger().setLevel('ERROR')
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python -

Retrieving speedtest.net configuration...
Testing from VNPT (14.177.252.75)...
Retrieving speedtest.net server list...
Selecting best server based on ping...
Hosted by Viettel IDC (Vinh) [261.88 km]: 10.311 ms
Testing download speed................................................................................
Download: 184.35 Mbit/s
Testing upload speed......................................................................................................
Upload: 202.71 Mbit/s


In [2]:
TRAIN_PATH = 'path/to/combined/dataset/Images/Train'
VALIDATE_PATH = 'path/to/combined/dataset/Images/Validate'
TEST_PATH = 'path/to/combined/dataset/Images/Test'

In [3]:
import os
PATH = 'Models/ResNet152V2'

BASE_MODEL_BEST = os.path.join(PATH, 'base_model_best.keras')
BASE_MODEL_TRAINED = os.path.join(PATH, 'base_model_trained.keras')
BASE_MODEL_FIG = os.path.join(PATH, 'base_model_fig.jpg')

FINE_TUNE_MODEL_BEST = os.path.join(PATH, 'fine_tune_model_best.keras')
FINE_TUNE_MODEL_TRAINED = os.path.join(PATH, 'fine_tune_model_trained.keras')
FINE_TUNE_MODE_FIG = os.path.join(PATH, 'fine_tune_model_fig.jpg')

In [4]:
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 128

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
validate_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

In [7]:
generated_train_data = train_generator.flow_from_directory(TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_validate_data = validate_generator.flow_from_directory(VALIDATE_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE)

Found 18751 images belonging to 38 classes.
Found 2757 images belonging to 38 classes.
Found 5169 images belonging to 38 classes.


In [8]:
CLASSES = 38
INITIAL_EPOCHS = 15
FINE_TUNE_EPOCHS = 15
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
FINE_TUNE_AT = 516

In [9]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [10]:
pretrained_model = ResNet152V2(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)


In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
base_checkpointer = ModelCheckpoint(
    filepath=BASE_MODEL_BEST
    verbose=1
)

fine_tune_checkpointer = ModelCheckpoint(
    filepath=FINE_TUNE_MODEL_BEST, 
    save_best_only=True,
    verbose=1, 
)


# Stop if no improvement after 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [12]:
import os

os.makedirs('Models/ResNet152V2', exist_ok=True)

In [13]:
for layer in pretrained_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit(
    generated_train_data,
    validation_data = generated_validate_data,
    validation_steps = generated_validate_data.n // BATCH_SIZE,
    steps_per_epoch = generated_train_data.n // BATCH_SIZE,
    callbacks = [base_checkpointer, early_stopping],
    epochs = INITIAL_EPOCHS,
    verbose = 1,
)
model.save(BASE_MODEL_TRAINED)

Epoch 1/15


c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 55/146 ━━━━━━━━━━━━━━━━━━━━ 33:44 22s/step - accuracy: 0.2474 - loss: 2.9693

c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step - accuracy: 0.3447 - loss: 2.4622 
Epoch 1: val_loss improved from inf to 1.41906, saving model to Models/ResNet152V2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 3788s 26s/step - accuracy: 0.3453 - loss: 2.4589 - val_accuracy: 0.5796 - val_loss: 1.4191
Epoch 2/15
  1/146 ━━━━━━━━━━━━━━━━━━━━ 38:06 16s/step - accuracy: 0.5469 - loss: 1.6239

c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 1.41906
146/146 ━━━━━━━━━━━━━━━━━━━━ 24s 59ms/step - accuracy: 0.5469 - loss: 1.6239 - val_accuracy: 0.5217 - val_loss: 1.6660
Epoch 3/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.5845 - loss: 1.3994 
Epoch 3: val_loss improved from 1.41906 to 1.29527, saving model to Models/ResNet152V2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 3664s 25s/step - accuracy: 0.5846 - loss: 1.3993 - val_accuracy: 0.6131 - val_loss: 1.2953
Epoch 4/15
  1/146 ━━━━━━━━━━━━━━━━━━━━ 1:11:06 29s/step - accuracy: 0.5938 - loss: 1.2649
Epoch 4: val_loss improved from 1.29527 to 1.22025, saving model to Models/ResNet152V2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 50s 143ms/step - accuracy: 0.5938 - loss: 1.2649 - val_accuracy: 0.6667 - val_loss: 1.2203
Epoch 5/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.6200 - loss: 1.2513 
Epoch 5: val_loss improved from 1.22025 to 1.17931, saving model to Models/ResNet152V2\base_model_best.keras
1

In [15]:
print(generated_train_data.image_shape)
print(generated_train_data.class_indices)

(300, 300, 3)
{'Banh beo': 0, 'Banh bot loc': 1, 'Banh can': 2, 'Banh canh': 3, 'Banh chung': 4, 'Banh cuon': 5, 'Banh duc': 6, 'Banh gio': 7, 'Banh khot': 8, 'Banh mi': 9, 'Banh pia': 10, 'Banh tet': 11, 'Banh trang nuong': 12, 'Banh xeo': 13, 'Bun bo Hue': 14, 'Bun dau mam tom': 15, 'Bun mam': 16, 'Bun rieu': 17, 'Bun thit nuong': 18, 'Bánh cu đơ': 19, 'Bánh mì cay': 20, 'Bánh đa cua': 21, 'Bánh đậu xanh': 22, 'Bò bía': 23, 'Bún cá': 24, 'Ca kho to': 25, 'Canh chua': 26, 'Cao lau': 27, 'Chao long': 28, 'Com tam': 29, 'Cơm cháy': 30, 'Goi cuon': 31, 'Hu tieu': 32, 'Mi quang': 33, 'Nem chua': 34, 'Nem nướng': 35, 'Pho': 36, 'Xoi xeo': 37}


In [16]:
for layer in pretrained_model.layers[:FINE_TUNE_AT]: layer.trainable = False
for layer in pretrained_model.layers[FINE_TUNE_AT:]: layer.trainable = True

In [17]:
from tensorflow.keras.optimizers import SGD
model.compile(
    optimizer = SGD(learning_rate=1e-4, momentum=0.9), 
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)

In [18]:
history_fine = model.fit(
    generated_train_data,
    validation_data = generated_validate_data,
    validation_steps = generated_validate_data.n // BATCH_SIZE,
    steps_per_epoch = generated_train_data.n // BATCH_SIZE,
    epochs = TOTAL_EPOCHS,
    initial_epoch = history.epoch[-1],
    callbacks = [fine_tune_checkpointer, early_stopping],
    verbose = 1,
)
model.save(FINE_TUNE_MODEL_TRAINED)

Epoch 9/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 25s/step - accuracy: 0.5182 - loss: 1.7415 
Epoch 9: val_loss improved from inf to 1.11962, saving model to Models/ResNet152V2\fine_tune_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 4345s 30s/step - accuracy: 0.5186 - loss: 1.7399 - val_accuracy: 0.6819 - val_loss: 1.1196
Epoch 10/30
  1/146 ━━━━━━━━━━━━━━━━━━━━ 1:22:53 34s/step - accuracy: 0.7031 - loss: 1.0933
Epoch 10: val_loss did not improve from 1.11962
146/146 ━━━━━━━━━━━━━━━━━━━━ 51s 112ms/step - accuracy: 0.7031 - loss: 1.0933 - val_accuracy: 0.6667 - val_loss: 1.5291
Epoch 11/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 33s/step - accuracy: 0.6629 - loss: 1.1745 
Epoch 11: val_loss improved from 1.11962 to 1.06462, saving model to Models/ResNet152V2\fine_tune_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 5206s 35s/step - accuracy: 0.6630 - loss: 1.1742 - val_accuracy: 0.6886 - val_loss: 1.0646
Epoch 11: early stopping


In [19]:
loss, accuracy = model.evaluate(generated_test_data)
print('Test accuracy:', accuracy)

162/162 ━━━━━━━━━━━━━━━━━━━━ 1063s 7s/step - accuracy: 0.6897 - loss: 1.0454
Test accuracy: 0.6889146566390991


In [23]:
from sklearn.metrics import classification_report
import numpy as np


# Sử dụng generator để dự đoán nhãn cho dữ liệu kiểm tra
generated_test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, shuffle=False)

# Dự đoán nhãn
predictions = model.predict(generated_test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = generated_test_data.classes

# Tính toán và in ra các chỉ số
class_labels = list(generated_test_data.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)

Found 5169 images belonging to 38 classes.


c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 883s 22s/step
                  precision    recall  f1-score   support

        Banh beo       0.84      0.71      0.77       129
    Banh bot loc       0.60      0.61      0.60       144
        Banh can       0.83      0.69      0.75       149
       Banh canh       0.44      0.37      0.40       193
      Banh chung       0.80      0.77      0.79       102
       Banh cuon       0.70      0.68      0.69       228
        Banh duc       0.41      0.20      0.27       133
        Banh gio       0.75      0.81      0.78       129
       Banh khot       0.69      0.83      0.76       167
         Banh mi       0.92      0.91      0.91       268
        Banh pia       0.86      0.84      0.85        89
        Banh tet       0.83      0.73      0.78       138
Banh trang nuong       0.90      0.75      0.82       159
        Banh xeo       0.81      0.83      0.82       235
      Bun bo Hue       0.54      0.68      0.60       306
 Bun dau mam tom       0.90   